In [19]:
import os
import sys
from dataclasses import dataclass
from pathlib import Path
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf 
from cnnClassifier.utils.common import logging,CustomException

In [5]:
# os.chdir('d:\\codes\\DeepLearning_Proj\\proj1\\research')
curr_dir = os.getcwd()
print(curr_dir)

d:\codes\DeepLearning_Proj\proj1\research


In [6]:
os.chdir(curr_dir)

In [7]:
os.chdir('../') # MOVING ONE LEVEL UP
print(os.getcwd())

d:\codes\DeepLearning_Proj\proj1


In [12]:
@dataclass(frozen = True)
class PrepareBaseModelConfig:
    root_dir                  : Path
    base_model_path           : Path
    update_base_model_path    : Path
    params_image_size         : list
    params_learning_rate      : float
    params_include_top        : bool
    params_weights            : str
    params_classes            : int

In [13]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directory

In [79]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directory([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model

        create_directory([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir                   = Path(config.root_dir),
            base_model_path            = Path(config.base_model_path),
            update_base_model_path     = Path(config.update_base_model_path),
            params_image_size          = self.params.IMAGE_SIZE,
            params_learning_rate       = self.params.LEARNING_RATE,
            params_include_top         = self.params.INCLUDE_TOP,
            params_weights             = self.params.WEIGHTS,
            params_classes             = self.params.CLASSES
        )
        
        return prepare_base_model_config


freeze_all = IF (TRUE) -> WE WANT THE MODEL TO FREEZE ALL THE MODEL'S WEIGHTS, WE DO NOT WANT TO RETRAIN THE MODEL FROM SCRATCH. THIS IS DONE
             SO THAT WE CAN USE VGG16 AS A FEATURE EXTRACTOR WITHOUT FUTURE TRAINING
             
freeze_till = AN INTEGER TO INDICATE HOW MANY LAYERS TO FREEZE FROM THE TOP (EXCLUDING THE LAST LAYER) THAT WE WANT TO FREEZE, THE REST WILL BE FINE TUNED BY US THIS IS KNOWN AS PARTIAL FINE TUNING WHEN WE WANT TO ADAPT THE MODEL TO A NEW TASK WITHOUT RETRAINING THE ENTIRE NETWORK.

THE EARLIER LAYERS ARE FROZEN BECAUSE THEY CONTAIN MORE GENERAL FEATURES, GENERAL FEATURES ARE LIKE THE FUNDAMENTAL BUILDING BLOCK THAT A MODEL LEARNS FROM THE DATA AT THE STARTING. THESE FUNDAMENTAL FEATURES CAN BE EDGE AND TEXTURES, BASIC SHAPES ETC.

tf.keras.models.Model() -> THIS FUNCTION IS USED TO CREATE NEW MODEL ITS LIKE ASSEMBLING DIFFERENT PIECES OF A PUZZLE

In [88]:
class PrepareBaseModel:
    def __init__(self, config:PrepareBaseModelConfig):
        self.config = config

    # THIS IS SAVING THE BASE MODEL THAT IS VGG16
    def get_base_model(self):
        # BELOW THIS self.model WILL BE PASSED INTO THE _prepare_full_model METHOEDc
        self.model        = tf.keras.applications.vgg16.VGG16(
            input_shape   = self.config.params_image_size,
            weights       = self.config.params_weights,
            include_top   = self.config.params_include_top
        )
        
        self.save_model(path  = self.config.base_model_path,
                        model = self.model)

    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:      
            for layer in model.layers:
                model.trainable = False

        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:  # -freeze_till WILL FREEZE THE STARTING LAYERS AND NOT INCLUDE THE LAST -freeze_till layes
                model.trainable = False                # WE FREEZE THE SELECTED LAYERS MEANING THAT THEIR WEIGHTS WILL NOT BE UPDATED DURING TRAINING
        
        # IT TAKES THE 2D FEATURE MAP AND CONVERT IT INTO 1D VECTOR THE RESULT IS STORED IN A VARIABLE
        # (model.output) -> IT MEANS THAT WE WANT TO FLATTEN THE OUTPUT PRODUCED BY THE CNN
        flatten_in = tf.keras.layers.Flatten()(model.output) 

        # THIS IS THE FULLEY CONNECTED LAYER THE OUTPUT WILL BE DEPENDANT ON THE NUMBER OF OUTPUT WE HAVE (CLASSES)
        prediction = tf.keras.layers.Dense(
            units = classes,
            activation ='softmax'
        )(flatten_in)

        # THE PROVIDED CODE CREATES A NEW MODEL BY COMBINING TWO PARTS OF THE NEURAL NETWORK 
        full_model = tf.keras.models.Model(
            inputs = model.input,
            outputs = prediction
        )

        full_model.compile(
            optimizer = tf.keras.optimizers.SGD(learning_rate = learning_rate),
            loss = tf.keras.losses.CategoricalCrossentropy(),
            metrics = ['accuracy']
        )

        full_model.summary()
        return full_model

    def update_base_model(self):
        # THESE ARE THE PARAMETERS THAT WE ARE PASSING IN VGG16 IN THE ABOVE FUNCTION THAT WILL RETURN THE SUMMARY OF THE MODEL
        self.full_model    = self._prepare_full_model(
            model          = self.model,
            classes        = self.config.params_classes,
            freeze_all     = True,
            freeze_till    = None,
            learning_rate  = self.config.params_learning_rate
        )

        # AFTER WE RECEIVE THE SUMMARY OF THE MODEL WE WILL SAVE IT
        self.save_model(path  = self.config.update_base_model_path,
                        model = self.full_model)
        
    @staticmethod
    def save_model(path : Path, model: tf.keras.Model):
        print(model.summary)
        model.save(path)

CREATING THE PIPELINE

In [89]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    CustomException(e,sys)

[2023-10-27 15:04:23,972] 31 root - INFO - YAML FILE config\config.yaml LOADED SUCCESSFULLY
[2023-10-27 15:04:23,973] 31 root - INFO - YAML FILE params.yaml LOADED SUCCESSFULLY
[2023-10-27 15:04:23,974] 50 root - INFO - CREATED DIRECTORY AT : artifacts
[2023-10-27 15:04:23,975] 50 root - INFO - CREATED DIRECTORY AT : artifacts/prepare_base_model
1
save models
<bound method Model.summary of <keras.src.engine.functional.Functional object at 0x00000195A8FA11D0>>
[2023-10-27 15:04:24,424] 359 tensorflow - WARNING - Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
2
update_base_model
Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_34 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (N

d:\codes\DeepLearning_Proj\proj1\DLproj1\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
